In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
import os
prefix = '/content/gdrive/My Drive/'
# modify "customized_path_to_your_homework" here to where you uploaded your homework
customized_path_to_your_homework = 'IDLSProject-main-2'
sys_path = os.path.join(prefix, customized_path_to_your_homework)
sys.path.append(sys_path)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
%cd '/content/gdrive/My Drive/IDLSProject-main'

/content/gdrive/My Drive/IDLSProject-main


In [ ]:
!pwd

/content/gdrive/My Drive/IDLSProject-main


In [ ]:
from torchsummary import summary

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import torch
import torch.utils.data
from sklearn.model_selection import train_test_split


data_dir = './data/pytorch'
with open(os.path.join(data_dir, 'word_dict_qrnn.pkl'), "rb") as f:
    word_dict = pickle.load(f)

train = pd.read_csv(os.path.join(data_dir, 'train_qrnn.csv'), header=None, names=None)
test_sample = pd.read_csv(os.path.join(data_dir, 'test_qrnn.csv'), header=None, names=None)

from sklearn.model_selection import train_test_split
test, val = train_test_split(test_sample, test_size=0.5)
train.shape, test.shape, val.shape
import torch
import torch.utils.data

# Turn the input pandas dataframe into tensors
train_y = torch.from_numpy(train[[0]].values).float()
train_X = torch.from_numpy(train.drop([0, 1], axis=1).values).long()

# Build the dataset
train_ds = torch.utils.data.TensorDataset(train_X, train_y)
# Build the dataloader
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=50)

######val data
# Turn the input pandas dataframe into tensors
val_y = torch.from_numpy(val[[0]].values).float()
val_X = torch.from_numpy(val.drop([0, 1], axis=1).values).long()

# Build the dataset
val_ds = torch.utils.data.TensorDataset(val_X, val_y)
# Build the dataloader
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=50)


#### Test data
# Turn the input pandas dataframe into tensors
test_y = torch.from_numpy(test[[0]].values).float()
test_X = torch.from_numpy(test.drop([0, 1], axis=1).values).long()

# Build the dataset
test_ds = torch.utils.data.TensorDataset(test_X, test_y)
# Build the dataloader
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=50)
print(test_y.shape)


torch.Size([10000, 1])


In [ ]:
!pip install sklearn

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 10
batch_size = 50
learning_rate = 0.001
embed_size = 300
seq_len = 500
dropout = 0.5
filter_size = 100
vocab_size = 5000
embed_dims = 32
hidden_size = 100
kernel_size = [3,4,5]
criterion = torch.nn.BCELoss().to(device)

In [ ]:
data_dir = './lstm-cnn-imdb-hyperband-trails/' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

filename = ""
def write_to_csv(trail_num, epochs, train_loss, train_acc, val_loss, val_acc, time_train):
    global filename
    filename = "./lstm-cnn-imdb-hyperband-trails/"+str(trail_num)+".csv"
    epoch = [i for i in range(epochs)]
    df_metrics = pd.DataFrame(list(zip(epoch, train_loss, train_acc, val_loss, val_acc, time_train)), columns =['Epoch', 'train_loss', 'train_acc', 'val_loss', 'val_acc', 'train_time'])
    df_metrics.to_csv(filename, index=False)    
    
def append_to_csv(epochs, accuracy):
    acc = [accuracy for i in range(epochs)]
    df_csv = pd.read_csv(filename)
    df_csv['Test_Accuracy']  = acc
    df_csv.to_csv(filename, index=False)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
class Combine(nn.Module):
    def __init__(self,trial,vocab_size, embed_size, filter_size, kernel_size, dropout, seq_len,hidden_size):
        super(Combine, self).__init__()
        dropout = trial.suggest_uniform("dropout",0.1, 0.6)
        hidden_size = trial.suggest_int("hidden_size",32,256)
        embed_size = trial.suggest_int("embed_size",16,128)
        filter_size = trial.suggest_int("filter_size",50,200)
        self.embedding = nn.Embedding(vocab_size, embed_size, padding_idx=0)
        self.lstm = nn.LSTM(input_size = embed_size, hidden_size = hidden_size,num_layers=1,batch_first=True)
        self.conv1 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[0],hidden_size])
        self.conv2 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[1],hidden_size])
        self.conv3 = torch.nn.Conv2d(1,filter_size,kernel_size=[kernel_size[2],hidden_size])
        self.mp1 = torch.nn.MaxPool1d(seq_len+1-kernel_size[0])
        self.mp2 = torch.nn.MaxPool1d(seq_len+1-kernel_size[1])
        self.mp3 = torch.nn.MaxPool1d(seq_len+1-kernel_size[2])
        self.dropout = torch.nn.Dropout(dropout)
        self.dense = nn.Linear(3*filter_size,1)
        self.sig = nn.Sigmoid()
        self.word_dict = None


    def forward(self, x,target):
    
        embeds = self.embedding(x)
        # print('embedds')
        # print(embeds.shape)
        lstm_out, _ = self.lstm(embeds)
        # print('lstm output')
        # print(lstm_out.shape)
        lstm_out = lstm_out.clone()
        lstm_out.unsqueeze_(1)
        # print('lstm output after')
        # print(lstm_out.shape)
        x1 = torch.tanh(self.dropout(self.conv1(lstm_out))).squeeze(3)
        x2 = torch.tanh(self.dropout(self.conv2(lstm_out))).squeeze(3)
        x3 = torch.tanh(self.dropout(self.conv3(lstm_out))).squeeze(3)
        # print('x1.shape')
        # print(x1.shape)
        f1 = self.mp1(x1).squeeze(2)
        # print('f1')
        # print(f1.shape)
        f2 = self.mp2(x2).squeeze(2)
        # print('f2')
        # print(f2.shape)
        f3 = self.mp3(x3).squeeze(2)
        # print('f3')
        # print(f3.shape)
        hidden = torch.cat([f1,f2,f3],dim=1)
        logits = self.dense(hidden)
        prediction = torch.sigmoid(logits)
        target = target.view([-1,1])
        correct_pred = torch.eq(torch.round(prediction).type(target.type()),target)
        accuracy = torch.sum(correct_pred)
        return prediction, accuracy
        # return self.sig(out)

In [ ]:
import time
import numpy as np
import torch.optim as optim

def objective(trial):
  optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-0)
  #momentum = trial.suggest_uniform("momentum", 0.0, 1.0)
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model = Combine(trial,vocab_size, embed_dims, filter_size, kernel_size, dropout, seq_len,hidden_size).to(device).to(device)
  trial.set_user_attr(key="best_model", value=model)
  optimizer = optim.Adam(model.parameters())
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  counter = 0
  CNN_acc = []
  CNN_valacc = []
  train_loss_epoch = []
  train_acc_epoch = []
  val_loss_epoch = []
  val_acc_epoch = []
  time_epoch = []
  model.train()
  final_val_acc = 0
  for e in range(epochs):
    start_time = time.time()
    train_loss = []
    train_acc = []
    for inputs, labels in train_dl:
        inputs, labels = inputs.cuda(), labels.cuda()
        model.zero_grad()
        logits, accuracy = model(inputs,labels)
        loss = criterion(logits,labels.float())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5)
        optimizer.step()
        train_loss.append(loss.item())
        train_acc.append(accuracy.item()*100/batch_size)
        if counter%100==0:
            print("Epoch: {}/{}".format(e,epochs),
                         "\tIteration: {}".format(counter),
                         "\t\tTrain Loss: {:.3f}".format(loss.item()),
                         "\tTrain Accuracy: {:.2f}".format(accuracy.item()*100/batch_size))
            CNN_acc.append(accuracy.item()*100/batch_size)
        counter += 1
    train_loss_epoch.append(np.round(np.mean(train_loss), 3))
    train_acc_epoch.append(np.round(np.mean(train_acc), 3))
    print("\tTrain Loss: {:.3f}".format(np.mean(train_loss)), "\tTrain Acc: {:.3f}".format(np.mean(train_acc)))

    with torch.no_grad():
        model.eval()
        val_acc = []
        val_loss = []
        for inputs, labels in val_dl:
            inputs_val, labels_val = inputs.cuda(), labels.cuda()
            logits_val,accuracy_val = model(inputs_val,labels_val)
            loss_val = criterion(logits_val,labels_val.float())
            val_acc.append(accuracy_val.item()*100/batch_size)
            val_loss.append(loss_val.item())
        val_loss_epoch.append(np.round(np.mean(val_loss), 3))
        val_acc_epoch.append(np.round(np.mean(val_acc), 3))
        final_val_acc = np.round(np.mean(val_acc), 3)
        print("\t\tVal Loss: {:.3f}".format(np.mean(val_loss)), "\t\tVal Acc: {:.3f}".format(np.mean(val_acc)))
        CNN_valacc.append(np.mean(val_acc))
        model.train()
        
    end_time = time.time()-start_time
    print("Time to train epoch: {0} s".format(end_time))
    time_epoch.append(np.round(end_time, 3))
  write_to_csv(trial.number,epochs, train_loss_epoch, train_acc_epoch, val_loss_epoch, val_acc_epoch, time_epoch)
  return final_val_acc

In [ ]:
def callback(study, trial):
    if study.best_trial.number == trial.number:
        study.set_user_attr(key="best_model", value=trial.user_attrs["best_model"])

In [ ]:
!pip install optuna

In [ ]:
import optuna
from optuna.trial import TrialState
study = optuna.create_study(direction="maximize",pruner=optuna.pruners.HyperbandPruner(
        min_resource=1, max_resource=epochs, reduction_factor=3
    ),)
study.optimize(objective, n_trials=10, callbacks=[callback])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of complete trials: ", len(complete_trials))
print("Best trial:")
trial = study.best_trial
best_model=study.user_attrs["best_model"]
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-05-15 05:40:29,120] A new study created in memory with name: no-name-73cb996c-9468-4fb6-9512-b1604b383aaa


Epoch: 0/10 	Iteration: 0 		Train Loss: 0.690 	Train Accuracy: 52.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.539 	Train Accuracy: 74.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.358 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.329 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.400 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.285 	Train Accuracy: 92.00
	Train Loss: 0.418 	Train Acc: 80.177
		Val Loss: 0.334 		Val Acc: 85.920
Time to train epoch: 25.41014266014099 s
Epoch: 1/10 	Iteration: 600 		Train Loss: 0.222 	Train Accuracy: 94.00
Epoch: 1/10 	Iteration: 700 		Train Loss: 0.207 	Train Accuracy: 92.00
Epoch: 1/10 	Iteration: 800 		Train Loss: 0.221 	Train Accuracy: 90.00
Epoch: 1/10 	Iteration: 900 		Train Loss: 0.193 	Train Accuracy: 92.00
Epoch: 1/10 	Iteration: 1000 		Train Loss: 0.285 	Train Accuracy: 90.00
Epoch: 1/10 	Iteration: 1100 		Train Loss: 0.167 	Train Accuracy: 94.00
	Train Loss: 0.263 	Train Acc: 89

[I 2022-05-15 05:44:15,033] Trial 0 finished with value: 87.22 and parameters: {'optimizer': 'RMSprop', 'lr': 0.14582249282326384, 'dropout': 0.38140377451464, 'hidden_size': 56, 'embed_size': 119, 'filter_size': 172}. Best is trial 0 with value: 87.22.


		Val Loss: 0.776 		Val Acc: 87.220
Time to train epoch: 22.02049708366394 s
Epoch: 0/10 	Iteration: 0 		Train Loss: 0.703 	Train Accuracy: 50.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.506 	Train Accuracy: 72.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.363 	Train Accuracy: 86.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.357 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.362 	Train Accuracy: 88.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.255 	Train Accuracy: 88.00
	Train Loss: 0.420 	Train Acc: 79.980
		Val Loss: 0.341 		Val Acc: 85.890
Time to train epoch: 38.40507221221924 s
Epoch: 1/10 	Iteration: 600 		Train Loss: 0.234 	Train Accuracy: 92.00
Epoch: 1/10 	Iteration: 700 		Train Loss: 0.222 	Train Accuracy: 90.00
Epoch: 1/10 	Iteration: 800 		Train Loss: 0.190 	Train Accuracy: 92.00
Epoch: 1/10 	Iteration: 900 		Train Loss: 0.212 	Train Accuracy: 88.00
Epoch: 1/10 	Iteration: 1000 		Train Loss: 0.296 	Train Accuracy: 86.00
Epoch: 1/10 	Iteration: 1100

[I 2022-05-15 05:50:42,168] Trial 1 finished with value: 87.14 and parameters: {'optimizer': 'SGD', 'lr': 0.003104307025647728, 'dropout': 0.5033119659745244, 'hidden_size': 158, 'embed_size': 92, 'filter_size': 172}. Best is trial 0 with value: 87.22.


...

		Val Loss: 0.681 		Val Acc: 87.150
Time to train epoch: 42.33254933357239 s
Epoch: 0/10 	Iteration: 0 		Train Loss: 0.702 	Train Accuracy: 50.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.558 	Train Accuracy: 74.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.453 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.432 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.421 	Train Accuracy: 82.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.307 	Train Accuracy: 86.00
	Train Loss: 0.480 	Train Acc: 75.830
		Val Loss: 0.388 		Val Acc: 83.550
Time to train epoch: 28.494454383850098 s
Epoch: 1/10 	Iteration: 600 		Train Loss: 0.347 	Train Accuracy: 88.00
Epoch: 1/10 	Iteration: 700 		Train Loss: 0.279 	Train Accuracy: 86.00
Epoch: 1/10 	Iteration: 800 		Train Loss: 0.367 	Train Accuracy: 88.00
Epoch: 1/10 	Iteration: 900 		Train Loss: 0.245 	Train Accuracy: 86.00
Epoch: 1/10 	Iteration: 1000 		Train Loss: 0.290 	Train Accuracy: 86.00
Epoch: 1/10 	Iteration: 110

[I 2022-05-15 06:22:45,284] Trial 8 finished with value: 86.51 and parameters: {'optimizer': 'RMSprop', 'lr': 8.570418056447273e-05, 'dropout': 0.4321482988124934, 'hidden_size': 87, 'embed_size': 42, 'filter_size': 197}. Best is trial 0 with value: 87.22.


		Val Loss: 0.552 		Val Acc: 86.510
Time to train epoch: 28.518890857696533 s
Epoch: 0/10 	Iteration: 0 		Train Loss: 0.692 	Train Accuracy: 52.00
Epoch: 0/10 	Iteration: 100 		Train Loss: 0.576 	Train Accuracy: 64.00
Epoch: 0/10 	Iteration: 200 		Train Loss: 0.434 	Train Accuracy: 80.00
Epoch: 0/10 	Iteration: 300 		Train Loss: 0.464 	Train Accuracy: 78.00
Epoch: 0/10 	Iteration: 400 		Train Loss: 0.425 	Train Accuracy: 84.00
Epoch: 0/10 	Iteration: 500 		Train Loss: 0.341 	Train Accuracy: 90.00
	Train Loss: 0.471 	Train Acc: 76.907
		Val Loss: 0.381 		Val Acc: 83.580
Time to train epoch: 49.051559925079346 s
Epoch: 1/10 	Iteration: 600 		Train Loss: 0.319 	Train Accuracy: 86.00
Epoch: 1/10 	Iteration: 700 		Train Loss: 0.271 	Train Accuracy: 90.00
Epoch: 1/10 	Iteration: 800 		Train Loss: 0.255 	Train Accuracy: 88.00
Epoch: 1/10 	Iteration: 900 		Train Loss: 0.216 	Train Accuracy: 88.00
Epoch: 1/10 	Iteration: 1000 		Train Loss: 0.312 	Train Accuracy: 86.00
Epoch: 1/10 	Iteration: 11

[I 2022-05-15 06:30:55,573] Trial 9 finished with value: 86.41 and parameters: {'optimizer': 'RMSprop', 'lr': 7.887143619924302e-05, 'dropout': 0.4914025837493733, 'hidden_size': 220, 'embed_size': 45, 'filter_size': 131}. Best is trial 0 with value: 87.22.


		Val Loss: 0.925 		Val Acc: 86.410
Time to train epoch: 48.96814274787903 s
Study statistics: 
  Number of finished trials:  10
  Number of complete trials:  10
Best trial:
  Value:  87.22
  Params: 
    optimizer: RMSprop
    lr: 0.14582249282326384
    dropout: 0.38140377451464
    hidden_size: 56
    embed_size: 119
    filter_size: 172


In [ ]:
torch.save(best_model.state_dict(),"./lstm-cnn-imdb-hyperband.pth")

In [ ]:
criterion = torch.nn.BCELoss().to(device)
def test(model, test_dl, epochs):
    with torch.no_grad():
        model.eval()
        test_acc = []
        test_loss = []
        for inputs, labels in test_dl:
            input_test, labels_test = inputs.cuda(), labels.cuda()
            logits_test,accuracy_test = model(input_test,labels_test)
            loss_test = criterion(logits_test,labels_test.float())
            test_acc.append(accuracy_test.item()*100/batch_size)
            test_loss.append(loss_test.item())
        print("Test Loss: {:.3f}".format(np.mean(test_loss)), "\tTest Acc: {:.3f}".format(np.mean(test_acc)))

In [ ]:
test(best_model, test_dl, epochs)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:762: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at  ../aten/src/ATen/native/cudnn/RNN.cpp:926.)
  self.dropout, self.training, self.bidirectional, self.batch_first)


Test Loss: 0.754 	Test Acc: 86.680
